In [4]:
import os
import csv
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge

from python.mechanics import StrainCalculator, StressCalculator
from python.deformation import Deformer
from python.optimizers import GradientDescentOptimizer, BFGSOptimizer, LBFGSOptimizer
from python.utils import TensorConverter, EV_TO_GPA
from python.visualization import Visualizer
from python.zeroelasticity import ElasticConstantsSolver,ElasticConstantsWorkflow
from python.structure import Atom ,Cell
from python.potentials import EAMAl1Potential
# 配置日志记录
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger(__name__)


In [5]:

# 创建基本FCC铝晶胞
a0 = 4.046  # 铝的晶格常数，单位：Å
lattice_vectors = np.array([[a0, 0.0, 0.0], [0.0, a0, 0.0], [0.0, 0.0, a0]])

# FCC基胞原子位置（分数坐标）
fcc_positions = np.array(
    [
        [0.0, 0.0, 0.0],  # 顶角
        [0.5, 0.5, 0.0],  # 面心
        [0.5, 0.0, 0.5],
        [0.0, 0.5, 0.5],
    ]
)

# 创建原子列表
atoms = []
for i, pos in enumerate(fcc_positions):
    # 转换到笛卡尔坐标
    cart_pos = np.dot(pos, lattice_vectors)
    # 铝原子质量为26.98 amu
    atoms.append(Atom(id=i, symbol="Al", mass_amu=26.98, position=cart_pos))

# 创建基本晶胞
base_cell = Cell(lattice_vectors=lattice_vectors, atoms=atoms, pbc_enabled=True)

# 创建4x4x4超胞，总共256个原子
supercell = base_cell.build_supercell((4, 4, 4))

logger.info(f"Created supercell with {supercell.num_atoms} atoms")
logger.info(f"Initial cell volume: {supercell.volume:.3f} Å³")
logger.info(f"Initial lattice parameters: {np.diag(supercell.lattice_vectors)}")

INFO:__main__:Created supercell with 256 atoms
INFO:__main__:Initial cell volume: 4238.943 Å³
INFO:__main__:Initial lattice parameters: [16.184 16.184 16.184]


In [6]:
# 创建EAM势
eam_potential = EAMAl1Potential()

DEBUG:python.potentials:EAM Al1 Potential initialized with cutoff=6.5.


In [9]:
# 设置参数，例如：
delta = 0.1  # 最大应变大小
num_steps = 20 # 每个应变分量使用多少2*num_steps步
optimizer_type = "LBFGS" # 优化器类型
optimizer_params = {"maxiter": 10000, "gtol": 1e-6} # 可以根据需要调整优化器参数
from datetime import datetime

# 获取当前时间字符串
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

save_path = f"../output_new/test_fcc_{current_time}"

# 初始化工作流类
workflow = ElasticConstantsWorkflow(
    cell=supercell,
    potential=eam_potential,
    delta=delta,
    num_steps=num_steps,
    optimizer_type=optimizer_type,
    optimizer_params=optimizer_params,
    save_path=save_path
)

# 运行整个流程，返回拟合得到的弹性常数矩阵和R²值
C_in_GPa, r2 = workflow.run()

print("Fitted Elastic Constants (GPa):\n", C_in_GPa)
print("Overall R²:", r2)

DEBUG:python.structure:Applying deformation to lattice vectors and atomic positions.
DEBUG:python.structure:Updated lattice vectors:
[[ 1.47359579e+01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.61840000e+01  0.00000000e+00]
 [-1.47359579e-05  0.00000000e+00  1.61840000e+01]]
DEBUG:python.structure:Updated cell volume: 3859.6694417273234
DEBUG:python.potentials:Calculated EAM potential energy: -854.355907682047 eV.
DEBUG:python.mechanics:Energy after negative deformation: -854.355907682047
DEBUG:python.mechanics:dUdh[2, 0] = 2.2737367544323206e-07
DEBUG:python.mechanics:Calculating derivative for component (2, 1).
DEBUG:python.structure:Applying deformation to lattice vectors and atomic positions.
DEBUG:python.structure:Updated lattice vectors:
[[1.52470316e+01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.61840000e+01 0.00000000e+00]
 [1.52470316e-05 0.00000000e+00 1.61840000e+01]]
DEBUG:python.structure:Applying deformation to lattice vectors and atomic positions.
DEBUG

Fitted Elastic Constants (GPa):
 [[ 1.28466734e+02  7.47382592e+01  7.47382592e+01  5.42270613e-01
  -1.40900274e+10 -1.40897139e+10]
 [ 7.47382594e+01  1.28466732e+02  7.47382594e+01 -1.40900164e+10
   5.42946467e-01 -1.40897139e+10]
 [ 7.47382596e+01  7.47382598e+01  1.28466732e+02 -1.40900164e+10
  -1.40900274e+10  5.43636874e-01]
 [-9.94635365e-08 -1.02282770e-07  1.81226893e-07  7.47180722e+10
  -1.67714204e-03  8.54544662e-04]
 [ 5.05569611e-08 -3.22524599e-07  2.18515950e-07  2.59564064e-03
   7.47180832e+10 -8.53670936e-04]
 [-2.96282497e-08  1.87939947e-07 -4.53183923e-07 -2.59228193e-03
   1.67557011e-03  7.47177697e+10]]
Overall R²: 0.21835357069508088
